In [1]:
"""
EM GMM
"""

'\nEM GMM\n'

In [23]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sympy as sy
from scipy.stats import multivariate_normal

In [201]:
mean =np.array([[30,5],[5,30]],dtype=float)
cov  =np.array([[[0.4,0],[0,0.8]],[[8,0],[0,1]]],dtype=float)
cov[1]

array([[8., 0.],
       [0., 1.]])

In [204]:

data =np.zeros((100,2),dtype=float)
for i in range(100):
    index =np.random.randint(0,2)
    data[i,:]=np.random.multivariate_normal(mean[index],cov[index],1)

In [215]:
def GMM(data,k=2):
    [n,p]=data.shape
    gamma =np.random.normal(0,1,(n,k))
    mean =np.random.normal(0,1,(k,p))
    cov =np.random.normal(0,1,(k,p,p))

    a   =(1.0/k)*np.ones((k,1),dtype=float)
    res =np.zeros((n,k),dtype=float)
    for i in range(n):
        for j in range(k):
            res[i,j] =a[j,0]*multivariate_normal.pdf(data[i,:],mean=mean[j,:],cov=np.eye(2))
    
    
    res = np.multiply(res,(1.0/np.sum(res,axis=1)[:,np.newaxis]))
    
    gamma =res
    maxpass =10000
    steps =0
    
    mean[0]=gamma[:,0][:,np.newaxis].T.dot(data) / np.sum(gamma[:,0])
    cov[0] =gamma[:,0][:,np.newaxis].T.dot(np.square(data-mean[0]))
    a[0,0]= np.sum(gamma[:,0])/n
   
    gamma_last =np.zeros_like(gamma)
    while(steps<maxpass and np.linalg.norm(gamma-gamma_last)>1e-4):
        steps  =steps+1
        gamma_last =gamma
        for j in range(k):
            mean[j]=gamma[:,j][:,np.newaxis].T.dot(data) / np.sum(gamma[:,j])
            cov[j] = np.multiply(gamma[:,j][:,np.newaxis],(data-mean[j][np.newaxis,:])).T.dot((data-mean[j][np.newaxis,:]))\
            / np.sum(gamma[:,j])
            #print('cov')
            #print(cov[j])
            a[j,0]= np.sum(gamma[:,j])/n
        #print('cov')
        #print(cov)
        for i in range(n):
            for j in range(k):
                gamma[i,j] =a[j,0]*multivariate_normal.pdf(data[i,:],mean=mean[j,:],cov=cov[j]+1e-7*np.eye(p))
        gamma =np.multiply(gamma,(1.0/np.sum(gamma,axis=1)[:,np.newaxis]))
    
    return [a,mean,cov]
        
        

            
            
            
            
  

            
    

In [216]:
res =GMM(data,k=2)
print('a ',res[0])
print('mean ',res[1])
print('cov ',res[2])

a  [[0.47]
 [0.53]]
mean  [[30.03662723  4.86654988]
 [ 5.06722253 29.90095125]]
cov  [[[ 0.47887066 -0.09288308]
  [-0.09288308  0.73777426]]

 [[ 7.95173897 -0.26515207]
  [-0.26515207  0.76863232]]]
